## DEEPSPEED + SLRUM + SINGULARITY

In [ ]:
# 初始環境設定
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path
current_foldr=!pwd
current_foldr=current_foldr[0]
current_foldr

In [ ]:
# 套件
!pip install cohere gdown kaleido langchain openai pyngrok pypdf python-dotenv sentence-transformers tiktoken -q
!pip install accelerate bitsandbytes hf_transfer huggingface_hub optimum transformers==4.36.2 -q 
!pip install appdirs black black[jupyter] datasets fire loralib sentencepiece gradio==3.48.0 -q
!pip install fastapi jieba matplotlib nltk peft==0.7.0 protobuf pydantic rouge-chinese scipy sse-starlette trl==0.7.6 uvicorn -q 
!pip install deepspeed -q

In [1]:
%%bash
cat << \EOF >  demo.script
## Method5 DEEPSPEED + Torch.Distributed.Run (singularity/notebook)
export GPUS_PER_NODE=2
nodes=( $( scontrol show hostnames $SLURM_JOB_NODELIS ) )
nodes_array=($nodes)
export MASTER_ADDR=${nodes_array[0]}
export MASTER_PORT=29003
export DS_CONFIG="ds_config_zero3.json"
export MODEL_ID="/work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf"
export DATASET="alpaca_gpt4_zh"
export OUTPUT_DIR="saves/LLaMA2-7B-Chat/lora/03_sft"
export MAX_SAMPLE=5000


## CLEAN CACHE
ps -ef |grep  train | awk '{print $2}' | xargs kill -9
rm -rf /work/g00cjz00/github/LLaMA-Factory/${OUTPUT_DIR}

## RUN (chage kernel to Python3 or your Image kernel)
## If you use Image kernel, unmark next two line, and mark /work/opt/ohpc/Taiwania3/libs/singularity/3.10.2/bin/singularity
## If you use python kernel, mark next two line, and unmark /work/opt/ohpc/Taiwania3/libs/singularity/3.10.2/bin/singularity

/work/opt/ohpc/Taiwania3/libs/singularity/3.10.2/bin/singularity run --nv --cleanenv -B /work -B /work/g00cjz00/libraryFolder/S_work-genai11_pytorch_2.1.2-cuda11.8-cudnn8-devel_opencv/local/lib:/home/g00cjz00/.local/lib -B /work/g00cjz00/libraryFolder/S_work-genai11_pytorch_2.1.2-cuda11.8-cudnn8-devel_opencv/local/bin:/home/g00cjz00/.local/bin -B /work/u00cjz00/os/ubuntu/bin:/usr/ubuntu_bin /work/u00cjz00/nvidia/pytorch_2.1.2-cuda11.8-cudnn8-devel_opencv.sif \
bash -c "export PATH=\$PATH:\$HOME/.local/bin; \
    cd /work/g00cjz00/github/LLaMA-Factory; \
    python3 -m torch.distributed.run \
    --nproc_per_node ${GPUS_PER_NODE} \
    --nnodes ${SLURM_NNODES} \
    --node_rank ${SLURM_PROCID} \
    --master_addr ${MASTER_ADDR} \
    --master_port ${MASTER_PORT} \
    src/train_bash.py \
    --deepspeed ${DS_CONFIG} \
    --stage sft \
    --do_train \
    --model_name_or_path ${MODEL_ID} \
    --dataset ${DATASET} \
    --template default \
    --finetuning_type lora \
    --lora_target q_proj,v_proj \
    --output_dir path_to_sft_checkpoint \
    --overwrite_cache \
    --per_device_train_batch_size 32 \
    --gradient_accumulation_steps 4 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 100 \
    --learning_rate 2e-5 \
    --num_train_epochs 3.0 \
    --plot_loss \
    --fp16 \
    --max_samples ${MAX_SAMPLE} \
    --output_dir ${OUTPUT_DIR}
"
EOF


## SRUN
srun -A GOV109189 --partition=gp4d --mpi=pmi2 --gres=gpu:1 --nodes=2 -c 4 --ntasks-per-node=1 --job-name=run_llama -o genai_%j.out -e genai_%j.err bash demo.script

In [ ]:
%%bash
cat << \EOF >  demo.script
## Method5 DEEPSPEED + Torch.Distributed.Run (singularity/notebook)
export GPUS_PER_NODE=2
nodes=( $( scontrol show hostnames $SLURM_JOB_NODELIS ) )
nodes_array=($nodes)
export MASTER_ADDR=${nodes_array[0]}
export MASTER_PORT=29003
export DS_CONFIG="ds_config_zero3.json"
export MODEL_ID="/work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf"
export DATASET="alpaca_gpt4_zh"
export OUTPUT_DIR="saves/LLaMA2-7B-Chat/lora/03_sft"
export MAX_SAMPLE=5000


## CLEAN CACHE
ps -ef |grep  train | awk '{print $2}' | xargs kill -9
rm -rf /work/g00cjz00/github/LLaMA-Factory/${OUTPUT_DIR}

## RUN (chage kernel to Python3 or your Image kernel)
## If you use Image kernel, unmark next two line, and mark /work/opt/ohpc/Taiwania3/libs/singularity/3.10.2/bin/singularity
## If you use python kernel, mark next two line, and unmark /work/opt/ohpc/Taiwania3/libs/singularity/3.10.2/bin/singularity

/work/opt/ohpc/Taiwania3/libs/singularity/3.10.2/bin/singularity run --nv --cleanenv -B /work -B /work/g00cjz00/libraryFolder/S_work-genai11_pytorch_2.1.2-cuda11.8-cudnn8-devel_opencv/local/lib:/home/g00cjz00/.local/lib -B /work/g00cjz00/libraryFolder/S_work-genai11_pytorch_2.1.2-cuda11.8-cudnn8-devel_opencv/local/bin:/home/g00cjz00/.local/bin -B /work/u00cjz00/os/ubuntu/bin:/usr/ubuntu_bin /work/u00cjz00/nvidia/pytorch_2.1.2-cuda11.8-cudnn8-devel_opencv.sif \
bash -c "export PATH=\$PATH:\$HOME/.local/bin; \
    cd /work/g00cjz00/github/LLaMA-Factory; \
    python3 -m torch.distributed.run \
    --nproc_per_node ${GPUS_PER_NODE} \
    --nnodes ${SLURM_NNODES} \
    --node_rank ${SLURM_PROCID} \
    --master_addr ${MASTER_ADDR} \
    --master_port ${MASTER_PORT} \
    src/train_bash.py \
    --deepspeed ${DS_CONFIG} \
    --stage sft \
    --do_train \
    --model_name_or_path ${MODEL_ID} \
    --dataset ${DATASET} \
    --template default \
    --finetuning_type lora \
    --lora_target q_proj,v_proj \
    --output_dir path_to_sft_checkpoint \
    --overwrite_cache \
    --per_device_train_batch_size 32 \
    --gradient_accumulation_steps 4 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 100 \
    --learning_rate 2e-5 \
    --num_train_epochs 3.0 \
    --plot_loss \
    --fp16 \
    --max_samples ${MAX_SAMPLE} \
    --output_dir ${OUTPUT_DIR}
"
EOF


cat << \EOF >  demo.slurm
#!/work/u00cjz00/binary/bash5.0/bin/bash
#SBATCH -A GOV109189                                                    ### project number, Example MST109178
#SBATCH -J _t2demo_                                                     ### Job name, Exmaple jupyterlab
#SBATCH -p gp4d                                                         ### Partition Name, Example ngs1gpu
#SBATCH --nodes=8                                                       ### Nodes, Default 1, node number
#SBATCH --ntasks-per-node=1                                             ### Tasks, Default 1, per node tasks
#SBATCH -c 8                                                            ### Cores assigned to each task, Example 4
#SBATCH --gres=gpu:2                                                    ### GPU number, Example gpu:1
#SBATCH --time=0-1:00:00                                                ### Runnung time, days-hours:minutes:seconds or hours:minutes:seconds
#SBATCH -o genai_%j.out                                                 ### Log folder, Here %j is job ID
#SBATCH -e genai_%j.err                                                 ### Log folder, Here %j is job ID


srun --mpi=pmi2 bash demo.script